In [2]:
import numpy as np

In [29]:
class Noeud:
    def __init__(self,valeur, parents=[]):
        self.valeur = valeur.astype(np.float64)
        self.parents = parents

        self.grad = np.zeros_like(self.valeur) # dA

        self._backward = lambda : None

    def __add__(A, B):
        A_2 = A
        B_2 = B
        #broadcasting
        if(A.valeur.shape[0] != B.valeur.shape[0]):
            if (B.valeur.shape[0] == 1):
                p = A.valeur.shape[0]
                B_2 = Noeud(np.ones((p,1))) *B
            elif(A.valeur.shape[0] == 1):
                p = B.valeur.shape[0]
                A_2 = Noeud(np.ones((p,1))) *A
            else:
                print("erreur de dimension")
        elif(A.valeur.shape[1] != B.valeur.shape[1]):
            if (B.valeur.shape[1] == 1):
                p = A.valeur.shape[1]
                B_2 = B* Noeud(np.ones((p,1)).T) 
            elif(A.valeur.shape[1] == 1):
                p = B.valeur.shape[1]
                A_2 = A* Noeud(np.ones((p,1)).T)
            else:
                print("erreur de dimension")
        else:
            print("erreur de dimension")          

        C = Noeud(A_2.valeur + B_2.valeur, parents=[A_2,B_2])
        
        def _backward():
            A_2.grad += C.grad
            B_2.grad += C.grad
        
        C._backward = _backward
        return C
    
    def __mul__(A, B):
        C = Noeud(np.matmul(A.valeur,B.valeur), parents=[A,B])

        def _backward():
            A.grad += np.matmul(C.grad, B.valeur.T)
            B.grad += np.matmul(A.valeur.T, C.grad)
        
        C._backward = _backward
        return C
    
    def exp(A):
        C = Noeud(np.exp(A.valeur), parents=[A])

        def _backward():
            A.grad += np.multiply(C.valeur, C.grad)
        
        C._backward = _backward
        return C
    
    def clear_grads(sortie):
        a_visiter = sortie.parents.copy()
        while not len(a_visiter)== 0 :
            s = a_visiter.pop()
            s.grad = np.zeros_like(s.valeur)
            for parent in s.parents:
                if parent not in a_visiter:
                    a_visiter.append(parent)

    def somme_lignes(A):
        # revient à multiplier par J_p à droite
        C = A * Noeud(np.ones((A.valeur.shape[1], 1)))
        return C
    
    def somme_colonnes(A):
        #revient à multiplier par J_p.t à gauche
        C =  Noeud(np.ones((A.valeur.shape[0], 1)).T) * A
        return C
    
    def somme(A):
        C = A.somme_lignes()
        D = C.somme_colonnes()
        return D
               
    
    def backward(sortie):
        #remettre à 0 les gradients de tout les noeuds 
        sortie.clear_grads()

        dejaVu = set()
        L = []

        def parcours(s):
            dejaVu.add(s)

            for parent in s.parents:
                if parent not in dejaVu:
                    parcours(parent)
            
            # fin de traitement de s
            L.append(s)

        parcours(sortie)

        sortie.grad = np.array([[1]])
        for s in reversed(L):
            s._backward()



In [30]:
A = Noeud(np.array(([2,3], [1,0])))
B = Noeud(np.array(([1,2], [3,4])))
d = Noeud(np.array(([3], [2])))

C = A*B
E = C+d
J = E.somme()

In [31]:
print(J.valeur)

[[40.]]


In [32]:
J.backward()

In [35]:
A.grad

array([[3., 7.],
       [3., 7.]])